## Initialise RAG

In [1]:

from RAG.librarian import Librarian 

librarian = Librarian(librarian_LLM_model = "GEMINI")

# SELECT SPECIALIST DATABASE
librarian.select_specialist(specialist = "traveller", specialist_LLM_model = "GEMINI", )

# Ask librarian to get acquinted with the specialist database
librarian.Traveller.load_data_model(reembed = False,
                                    embed_id = 0,
                                    data_model_keys = {"TEST - CLIENT":"CLIENT ID",
                                                        "TEST - CLIENT REQUEST":"CLIENT ID",
                                                        "TEST - FLIGHTS":"FLIGHT ID",
                                                        "TEST - ACCOMODATIONS":"ACCOMODATION ID",
                                                        "TEST - ACTIVITIES":"ACTIVITY ID",
                                                        "TEST - SERVICES":"SERVICE ID",
                                                        },
                                    reembed_table = {"TEST - CLIENT":True,
                                                    "TEST - CLIENT REQUEST":True,
                                                    "TEST - FLIGHTS":True,
                                                    "TEST - ACCOMODATIONS":True,
                                                    "TEST - ACTIVITIES":True,
                                                    "TEST - SERVICES":True,
                                                    }
                                                    
                                    )


loading specialist: TRAVELLER ...
TRAVELLER embedding: TEST - CLIENT - LOADED
TRAVELLER embedding: TEST - CLIENT REQUEST - LOADED
TRAVELLER embedding: TEST - FLIGHTS - LOADED
TRAVELLER embedding: TEST - ACCOMODATIONS - LOADED
TRAVELLER embedding: TEST - ACTIVITIES - LOADED
TRAVELLER embedding: TEST - SERVICES - LOADED
TRAVELLER loaded


## _*Generate*_ : travel package from customer/agent prompt  + inventory

In [7]:
# Ask Traveller to generate a travel package
initial_query = "Hafeez from Kuala Lumpur wants to go Bali for 4 days for 2 pax. Budget: $2000. I want to chill at the beach"

convo_package = librarian.Traveller.III_generate_travel_package(initial_query = initial_query,
                                                                 topN = 6, 
                                                                 model_name = "gemini-pro",
                                                                 )

Token size of the prompt for cl100k_base ~ 3745
**Summary**

Immerse yourself in the beauty of Bali on this 4-day beach getaway designed to soothe your soul and rejuvenate your senses. Dive into the pristine waters, bask in the warm sun, indulge in luxurious spa treatments, and explore the island's stunning natural wonders. This trip_recommendation offers a perfect blend of relaxation, adventure, and cultural experiences, ensuring an unforgettable escape to paradise.

**Journey Highlights**

* Chill out on the white-sand beaches of Bali
* Pamper yourself with a rejuvenating spa treatment
* Embark on a scenic hiking and cliff-hunting tour
* Savor delicious meals at local restaurants
* Capture the beauty of Bali's rice fields
* Your journey takes you to: Kuala Lumpur - Denpasar Bali - Kuala Lumpur

**Itinerary**

**Day 1**

* 0900hrs: Depart from Kuala Lumpur International Airport (FLIGHT ID: 1)
* 1100hrs: Arrive at Denpasar Bali Airport (DPS)
* 1200hrs: Transfer to Hard Rock Hotel Bali 

## Regenerate : travel package from followup prompt + inventory

In [10]:
followup_query = "Give me the cheapest bali activities instead of that 4.5K hunting tour"

convo_package = librarian.Traveller.III_generate_travel_package(initial_query = "",
                                                                followup_query = followup_query,
                                                                 topN = 6, 
                                                                 model_name = "gemini-pro",
                                                                 )

Token size of the prompt for cl100k_base ~ 3857
**Refined Travel Package**

**Summary**

Immerse yourself in the beauty of Bali on this tailored 4-day beach getaway designed to soothe your soul and rejuvenate your senses. Dive into the pristine waters, bask in the warm sun, indulge in luxurious spa treatments, and explore the island's stunning natural wonders. This trip offers a perfect blend of relaxation, adventure, and cultural experiences, ensuring an unforgettable escape to paradise.

**Journey Highlights**

* Chill out on the white-sand beaches of Bali
* Pamper yourself with a rejuvenating spa treatment
* Embark on a scenic hiking and cliff-hunting tour
* Savor delicious meals at local restaurants
* Capture the beauty of Bali's rice fields
* Your journey takes you to: Kuala Lumpur - Denpasar Bali - Kuala Lumpur

**Itinerary**

**Day 1**

* 0900hrs: Depart from Kuala Lumpur International Airport (FLIGHT ID: 1)
* 1100hrs: Arrive at Denpasar Bali Airport (DPS)
* 1200hrs: Transfer to